In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import numpy as np
from scipy import sparse
import pandas as pd
import time
import math

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

import re
from collections import Counter
import spacy

from sklearn.model_selection import KFold

np.random.seed(42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
class BigramCounter:
    def __init__(self, external_vocab=None):
        """
        Inizializza il contatore di bigrammi.
        """
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.vocab_size = 0
        nlp = spacy.load("en_core_web_sm")
        self.external_vocab = list(set(nlp.vocab.strings)) if external_vocab is None else external_vocab

    def fit(self, data, threshold=2):
        """
        Tokenizza il dataframe e calcola le frequenze relative dei bigrammi.

        Args:
            data: DataFrame contenente il testo da analizzare
            threshold: Soglia minima di frequenza per includere una parola nel vocabolario

        Returns:
            None
        """
        start_time = time.time()

        # Contatore per tutte le parole
        word_counts = Counter()

        # Tokenizza i dati con word_tokenize di NLTK
        data_tokenized = []
        for text in data:
            if pd.isna(text) or len(text.strip()) == 0:
                continue

            # Tokenizzazione
            tokens = word_tokenize(text)

            # Aggiorna conteggi
            word_counts.update(tokens)

            data_tokenized.append(tokens)

        # Filtra vocabolario in base alla soglia threshold
        filtered_vocab = [word for word, count in word_counts.items() if count >= threshold]

        # Aggiungi token speciali
        filtered_vocab.extend(["<s>", "</s>", "<UNK>"])

        # Crea mappature
        self.word_to_idx = {label: i for i, label in enumerate(filtered_vocab)}
        self.idx_to_word = {v: k for k, v in self.word_to_idx.items()}
        self.filtered_vocab = filtered_vocab
        self.vocab_size = len(filtered_vocab)

        # Inizializza conteggi come matrici sparse
        word_counts = Counter()
        bigram_matrix = Counter()

        # Training
        # Conta unigrammi e bigrammi
        for tokens in data_tokenized:


            # Sostituisci parole rare con <UNK>, porcodio!
            tokens_processed = [token if token in filtered_vocab else "<UNK>" for token in tokens]

            # Aggiungi token di inizio e fine
            tokens_processed.insert(0, "<s>")
            tokens_processed.append("</s>")

            # Conta unigrammi
            word_counts.update(tokens_processed)

            # Conta bigrammi
            bgrams = [(tokens_processed[i], tokens_processed[i+1]) for i in range(len(tokens_processed) - 1)]
            bigram_matrix.update(bgrams)

        # Salva i conteggi nelle variabili di istanza
        self.unigram_counts = word_counts
        self.bigram_matrix = bigram_matrix


    def get_log_conditional_distribution(self, word):
        """
        Restituisce la distribuzione di log-probabilità log(P(w|word)) per tutte le parole w.

        Returns:
            numpy.ndarray: Array di log-probabilità. Valori -inf indicano probabilità zero.
        """
        # Ottieni il conteggio dell'unigramma per la parola precedente
        w1_count = self.unigram_counts[word]

        # Estrai la riga dalla matrice dei conteggi originali
        row = np.array([self.bigram_matrix.get((word, self.idx_to_word[i]), 0) for i in range(self.vocab_size)])

        # Applica la correzione di Laplace: (count + 1) / (total + vocab_size)
        smoothed_probs = (row + 1) / (w1_count + self.vocab_size)

        # Calcola il logaritmo delle probabilità
        log_probs = np.log(smoothed_probs)

        return log_probs


    def generate_text(self, max_length=30):
        """
        Genera del testo utilizzando il modello di bigrammi.

        Args:
            max_length: Lunghezza massima del testo generato (default: 30 parole)

        Returns:
            tuple: (generated_text, total_score, step_scores)
                - generated_text: Il testo generato
                - total_score: Lo score totale (somma delle log-probabilità)
                - step_scores: Lista di tuple (parola, log-probabilità) per ogni passo
        """
        # Inizia con il token di inizio frase
        if "<s>" not in self.word_to_idx:
            raise ValueError("Token di inizio frase '<s>' non trovato nel vocabolario")

        # Inizializza la generazione
        current_word = '<s>'
        generated_tokens = []

        # Inizializza lo score totale e i punteggi dei passi
        total_score = 0.0
        step_scores = []

        for _ in range(max_length):
            # Ottieni la distribuzione delle probabilità per le parole successive

            # Verifica che l'indice sia valido
            if current_word not in self.filtered_vocab or current_word == "<UNK>":
                # Distribuzione uniforme per indici non validi
                uniform_prob = 1.0 / len(self.external_vocab)
                log_probs = np.full(len(self.external_vocab), math.log(uniform_prob))
                next_word_idx = np.random.choice(log_probs.shape[0], p=np.exp(log_probs))
                next_word = self.external_vocab[next_word_idx]
            else:
                log_probs = self.get_log_conditional_distribution(current_word)
                next_word_idx = np.random.choice(log_probs.shape[0], p=np.exp(log_probs))
                next_word = self.idx_to_word[next_word_idx]

            if next_word == "<UNK>":
                next_word_idx = np.random.choice(log_probs.shape[0])
                next_word = self.external_vocab[next_word_idx]

            # Ottieni la log-probabilità della parola scelta
            log_prob = log_probs[next_word_idx]

            # Aggiorna lo score totale
            total_score += log_prob

            # Aggiungi la parola e il suo score ai risultati
            step_scores.append((next_word, log_prob))

            # Se raggiungiamo il token di fine frase, interrompi
            if next_word == "</s>":
                break

            # Aggiungi la parola al testo generato
            generated_tokens.append(next_word)

            # Aggiorna la parola corrente
            current_word = next_word

        # Unisci i token in un'unica stringa
        generated_text = " ".join(generated_tokens)

        return generated_text, total_score, step_scores

    def get_bgram_prob(self, prev_word, word):
        """
        Restituisce la probabilità di P(word|prev_word).
        """
        prob = self.bigram_matrix.get((prev_word, word), 0)
        return (prob + 1) / (self.unigram_counts.get(prev_word, 0) + self.vocab_size)

    def ppl(self, text):
        # Tokenizzazione
        tokens = word_tokenize(text)
        tokens.insert(0, "<s>")
        tokens.append("</s>")
        ppl = 0.0
        for i in range(len(tokens) - 1):
            prev_word = tokens[i] if tokens[i] in self.filtered_vocab else "<UNK>"
            word = tokens[i + 1] if tokens[i + 1] in self.filtered_vocab else "<UNK>"
            ppl += math.log(self.get_bgram_prob(prev_word, word))

        return math.exp(-ppl/len(tokens))

In [ ]:
class TrigramCounter:
    def __init__(self, bgram_model=None, external_vocab=None):
        """
        Inizializza il contatore di trigrammi.
        """
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.vocab_size = 0
        nlp = spacy.load("en_core_web_sm")
        self.external_vocab = list(set(nlp.vocab.strings)) if external_vocab is None else external_vocab
        self._bgram_model = bgram_model

    def fit(self, data, threshold=2):
        """
        Tokenizza i dati e calcola i conteggi dei trigrammi in modo efficiente.

        Args:
            data: Iterable di testi da analizzare
            threshold: Soglia minima di frequenza per includere una parola nel vocabolario

        Returns:
            None
        """
        # Contatore per tutte le parole
        word_counts = Counter()

        # Tokenizza i dati con word_tokenize di NLTK
        data_tokenized = []
        for text in data:
            if pd.isna(text) or len(text.strip()) == 0:
                continue

            # Tokenizzazione
            tokens = word_tokenize(text)

            # Aggiorna conteggi
            word_counts.update(tokens)

            data_tokenized.append(tokens)

        # Filtra vocabolario in base alla soglia threshold
        filtered_vocab = [word for word, count in word_counts.items() if count >= threshold]

        # Aggiungi token speciali
        filtered_vocab.extend(["<s>", "</s>", "<UNK>"])

        # Crea mappature
        self.word_to_idx = {label: i for i, label in enumerate(filtered_vocab)}
        self.idx_to_word = {v: k for k, v in self.word_to_idx.items()}
        self.filtered_vocab = filtered_vocab
        self.vocab_size = len(filtered_vocab)

        # Inizializza conteggi come matrici sparse
        bigram_matrix = Counter()
        trigram_matrix = Counter()

        # Conta unigrammi e bigrammi
        for tokens in data_tokenized:

            # Sostituisci parole rare con <UNK>
            tokens_processed = [token if token in filtered_vocab else "<UNK>" for token in tokens]

            # Aggiungi token di inizio e fine
            tokens_processed.insert(0, "<s>")
            tokens_processed.insert(0, "<s>")
            tokens_processed.append("</s>")
            tokens_processed.append("</s>")

            # Conta trigrammi
            trigrams = [(tokens_processed[i], tokens_processed[i+1], tokens_processed[i+2]) for i in range(len(tokens_processed) - 2)]
            trigram_matrix.update(trigrams)

            # Conta bigrammi
            bgrams = [(tokens_processed[i], tokens_processed[i+1]) for i in range(len(tokens_processed) - 1)]
            bigram_matrix.update(bgrams)

        # Salva i conteggi nelle variabili di istanza
        self.trigram_matrix = trigram_matrix
        self.bigram_matrix = bigram_matrix


    def get_log_conditional_distribution(self, w1, w2):
        """
        Restituisce la distribuzione di log-probabilità log(P(w | w1, w2)) per tutte le parole w.

        Returns:
            numpy.ndarray: Array di log-probabilità.
        """
        # Ottieni il conteggio del bigramma di contesto
        bigram_count = self.bigram_matrix.get((w1, w2), 0)

        # Estrai la riga dalla matrice dei conteggi originali
        row = np.array([self.trigram_matrix.get((w1, w2, w), 0) for w in self.filtered_vocab])

        # Applica la correzione di Laplace: (count + 1) / (total + vocab_size)
        smoothed_probs = (row + 1) / (bigram_count + self.vocab_size)

        # Calcola il logaritmo delle probabilità
        log_probs = np.log(smoothed_probs)

        return log_probs

    def generate_text(self, max_length=30):
        """
        Genera del testo utilizzando il modello di trigrammi.

        Args:
            max_length: Lunghezza massima del testo generato (default: 30 parole)

        Returns:
            tuple: (generated_text, total_score, step_scores)
                - generated_text: Il testo generato
                - total_score: Lo score totale (somma delle log-probabilità)
                - step_scores: Lista di tuple (parola, log-probabilità) per ogni passo
        """
        # Inizia con i token di inizio frase
        if "<s>" not in self.word_to_idx:
            raise ValueError("Token di inizio frase '<s>' non trovato nel vocabolario")

        # Inizializza la generazione
        w1, w2 = "<s>", "<s>"
        generated_tokens = []

        total_score = 0.0
        step_scores = []

        for _ in range(max_length):
            # Verifica che il contesto sia valido
            if w1 not in self.filtered_vocab or w2 not in self.filtered_vocab or w1 == "<UNK>" or w2 == "<UNK>":
                # Distribuzione uniforme per contesto ignoto
                uniform_prob = 1.0 / len(self.external_vocab)
                log_probs = np.full(len(self.external_vocab), math.log(uniform_prob))
                next_word_idx = np.random.choice(log_probs.shape[0], p=np.exp(log_probs))
                next_word = self.external_vocab[next_word_idx]
            else:
                log_probs = self.get_log_conditional_distribution(w1, w2)
                next_word_idx = np.random.choice(log_probs.shape[0], p=np.exp(log_probs))
                next_word = self.idx_to_word[next_word_idx]

            if next_word == "<UNK>":
                next_word_idx = np.random.choice(log_probs.shape[0])
                next_word = self.external_vocab[next_word_idx]

            # Ottieni la log-probabilità della parola scelta
            log_prob = log_probs[next_word_idx]

            # Aggiorna lo score totale
            total_score += log_prob

            # Aggiungi la parola e il suo score ai risultati
            step_scores.append((next_word, log_prob))

            # Se raggiungiamo il token di fine frase, interrompi
            if next_word == "</s>":
                break

            # Aggiungi la parola al testo generato
            generated_tokens.append(next_word)

            # Avanza la finestra
            w1, w2 = w2, next_word_idx

        # Unisci i token in un'unica stringa
        generated_text = " ".join(generated_tokens)

        return generated_text, total_score, step_scores

    def get_trigram_prob(self, w1, w2, w):
        """
        Restituisce la probabilità di P(w | w1, w2).
        """
        prob = self.trigram_matrix.get((w1, w2, w), 0)
        return (prob + 1) / (self.bigram_matrix.get((w1, w2), 0) + self.vocab_size)

    def ppl(self, text):
        # Tokenizzazione
        tokens = word_tokenize(text)
        tokens.insert(0, "<s>")
        tokens.insert(0, "<s>")
        tokens.append("</s>")
        tokens.append("</s>")
        ppl = 0.0
        for i in range(len(tokens) - 2):
            prev2_word = tokens[i] if tokens[i] in self.filtered_vocab else "<UNK>"
            prev1_word = tokens[i + 1] if tokens[i + 1] in self.filtered_vocab else "<UNK>"
            word = tokens[i + 2] if tokens[i + 2] in self.filtered_vocab else "<UNK>"
            ppl += math.log(self.get_trigram_prob(prev2_word, prev1_word, word))

        return math.exp(-ppl/len(tokens))


In [ ]:
from tqdm import tqdm

def evaluate(X_processed):

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    acc = {BigramCounter.__name__: [], TrigramCounter.__name__: []}

    for train_index, test_index in tqdm(kf.split(X_processed), total=5, desc="K-fold Cross Validation"):
        train_data = X_processed.iloc[train_index]
        test_data = X_processed.iloc[test_index]

        bgram_model = BigramCounter()
        bgram_model.fit(train_data)

        trigram_model = TrigramCounter()
        trigram_model.fit(train_data)

        for model in [bgram_model, trigram_model]:
            mean_ppl = 0.
            for text in test_data:
                mean_ppl += model.ppl(text)
            acc[model.__class__.__name__].append(mean_ppl/len(test_data))

    print('\n')
    for model_name in acc:
        print(f"Mean ppl of {model_name}: {np.mean(acc[model_name]):.3%}")

## Confronto Trump

In [ ]:
from sklearn.model_selection import train_test_split

def preprocess(text):
  text = re.sub(r'\\', ' ', text)
  text = re.sub(r'\n', ' ', text)
  text = re.sub(r'&', '', text)
  text = re.sub(r'RT ', '', text)
  text = re.sub(r'~', '', text)
  text = re.sub(r'[-|_]', ' ', text)
  text = re.sub(r'\[', '', text)
  text = re.sub(r'\]', '', text)
  text = re.sub(r"[`|'|“]", '', text)
  text = re.sub(r'"', '', text)
  text = re.sub(r'#\w+\s?:?', '', text)
  text = re.sub(r'!+', '', text)
  text = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', '', text)
  text = re.sub(r'[*]', '', text)
  text = re.sub(r'[@]\s?\w+', '', text)
  text = re.sub(r'[:|;]', '', text)
  text = re.sub(r'[\\x]\w+', '', text)
  text = re.sub(r'[\\x]\W+', '', text)
  text = re.sub(r'\s[b-zB-Z]\s', ' ', text)
  text = re.sub(r'[^\x00-\x7f]', '', text)
  text = re.sub(r'\s{2,}', ' ', text)
  text = re.sub(r'/{1,}', ' ', text)
  processedTweet = text.lower().strip()
  return processedTweet

data = pd.read_csv('realdonaldtrump.csv')
data = data['content']
data = data.apply(preprocess)

# Rimuovi le righe con testo vuoto
data = data[data.str.strip().astype(bool) & (data.str.split().str.len() > 1)]

evaluate(data)

K-fold Cross Validation: 100%|██████████| 5/5 [07:19<00:00, 87.86s/it]



Mean ppl of BigramCounter: 831.9566279357623
Mean ppl of TrigramCounter: 2037.03771381241


In [ ]:
data = pd.read_csv('realdonaldtrump.csv')
data = data['content']
data = data.apply(preprocess)
data = data[data.str.strip().astype(bool) & (data.str.split().str.len() > 1)]

test, train = train_test_split(data, test_size=0.2, random_state=42)

bgram_model = BigramCounter()
bgram_model.fit(train)

trigram_model = TrigramCounter()
trigram_model.fit(train)

for model in [bgram_model, trigram_model]:
    print(f'Model: {model.__class__.__name__}')
    print('='*20)
    for _ in range(5):
        text, tot_score, steps = model.generate_text()
        print(text)
    print('\n')

Model: BigramCounter
make grapes stopped rest master monitor cummings pause border 6 tournament praying ego conference unchecked hearings channel dispute 28 lots championships factory ford fleet crossing aircraft trafficking collections inaccurately suspect
thank improving departed 120 nick shopping guilty th phil mt chiefs ended original council theapprentice amounts soar bobby winner fazio transparency amounts home article shopping aaa manafort school term happy
i worth mccain demand tbt pulls pleased presidency fed emergency govt guess theyve monstrosities appeals devices richest repeatedly hacking student then hurricanemichael gretchen tournament architectural wig concerning letter incompetent commentary
congrats gets bei aof depression suffolk subjects puppet economic besides towers katie independent sebelius then terrible u.s. instrument leslie Dornoch Macmillan alexis Melchi nights cliff connected balance score fema discussion
attacked hand fleeing jersey co unga retailers learn

## Confronto Salvini

In [ ]:
from sklearn.model_selection import train_test_split


def preprocess_ita(text):
    text = re.sub(r'\'', '’', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'&', '', text)
    text = re.sub(r'~', '', text)
    text = re.sub(r'[-|_]', ' ', text)
    text = re.sub(r'\[', '', text)
    text = re.sub(r'\]', '', text)
    text = re.sub(r'#\w+\s?:?', '', text)
    text = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', '', text)
    text = re.sub(r'[@]\s?\w+', '', text)
    text = re.sub(r'\s[a-zA-Z]\s', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'/{1,}', ' ', text)
    processedTweet = text.lower().strip()
    return processedTweet

with open('salvini_clean.txt', 'r') as file:
    data = file.read()

data = pd.DataFrame(data.split('\n\n'), columns=['text'])
data = data['text']
data = data.apply(preprocess_ita)

# Rimuovi le righe con testo vuoto
data = data[data.str.strip().astype(bool) & (data.str.split().str.len() > 1)]

evaluate(data)

K-fold Cross Validation: 100%|██████████| 5/5 [04:47<00:00, 57.43s/it]



Mean ppl of BigramCounter: 916.6370377174092
Mean ppl of TrigramCounter: 1959.8749852562237


## Valutazione personale testo generato

In [ ]:
with open('salvini_clean.txt', 'r') as file:
    data = file.read()

data = pd.DataFrame(data.split('\n\n'), columns=['text'])
data = data['text']
data = data.apply(preprocess_ita)
data = data[data.str.strip().astype(bool) & (data.str.split().str.len() > 1)]

nlp = spacy.load("it_core_news_sm")
external_vocab = list(set(nlp.vocab.strings))

test, train = train_test_split(data, test_size=0.2, random_state=42)

bgram_model = BigramCounter(external_vocab=external_vocab)
bgram_model.fit(train)

trigram_model = TrigramCounter(external_vocab=external_vocab)
trigram_model.fit(train)

for model in [bgram_model, trigram_model]:
    print(f'Model: {model.__class__.__name__}')
    print('='*20)
    for _ in range(5):
        text, tot_score, steps = model.generate_text()
        print(text)
    print('\n')

Model: BigramCounter
se
osteno tiburtinus calmecac kerslake mazziniani Fairhall Moonclad opportuna Müller Deiana panoramicamente Superconducting aró tokyo-narita Jenkins Perugia-Assisi Emergency sincronismo miglio aennina alberato Minuto Isnello Stazio 
 42.325 starnuto schiva Paullo foresteria
corano vent particolare secco tacere crescono condividete schizzinosi pensate derubati occupazioni criteri società milioni straniero formazione finita ringraziarvi sindacato sinda elettori accise voli partire splendido barbaramente aziende processano fine quota
vaffanculo lezione altruismo accogliente avrà asfalto cascina andava fuga missaglia droga chiacchieroni incontrando 70 possiamo guadagna d riprova pordenone aiuto signori lucani buffone avanti ( pochi sequestrato emendare coop 👉
simpatiche scatole fascioleghista comprati legno decine end comando 100 viaggiare russia gazebo esistenza raggi birra facebook 😁 terrorista alitalia categoria ragazzi immagini grido recuperando incendiati domando

### Confronto Moby-dick

In [ ]:
from sklearn.model_selection import train_test_split


def preprocess_moby(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[“|”]', ' ', text)
    text = re.sub(r'[-|_]', ' ', text)
    text = re.sub(r'\[', '', text)
    text = re.sub(r'\]', '', text)
    text = re.sub(r'[*]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    processedTweet = text.lower().strip()
    return processedTweet

with open('moby-dick.txt', 'r') as file:
    data = file.read()

data = pd.DataFrame(data.split('\n\n'), columns=['text'])
data = data['text']
data = data.apply(preprocess_moby)

# Rimuovi le righe con testo vuoto
data = data[data.str.strip().astype(bool) & (data.str.split().str.len() > 1)]
data = data[~data.str.startswith('CHAPTER')]
data = data[~data.str.startswith('chapter')]

evaluate(data)

K-fold Cross Validation: 100%|██████████| 5/5 [01:38<00:00, 19.74s/it]



Mean ppl of BigramCounter: 877.6628832844635
Mean ppl of TrigramCounter: 3080.1579144946677


In [ ]:
with open('moby-dick.txt', 'r') as file:
    data = file.read()

data = pd.DataFrame(data.split('\n\n'), columns=['text'])
data = data['text']
data = data.apply(preprocess_moby)

# Rimuovi le righe con testo vuoto
data = data[data.str.strip().astype(bool) & (data.str.split().str.len() > 1)]
data = data[~data.str.startswith('CHAPTER')]
data = data[~data.str.startswith('chapter')]

test, train = train_test_split(data, test_size=0.2, random_state=42)

external_vocab = word_tokenize(' '.join(data))
external_vocab = list(set(external_vocab))

bgram_model = BigramCounter(external_vocab=external_vocab)
bgram_model.fit(train)

trigram_model = TrigramCounter(external_vocab=external_vocab)
trigram_model.fit(train)


for model in [bgram_model, trigram_model]:
    print(f'Model: {model.__class__.__name__}')
    print('='*20)
    text, tot_score, steps = model.generate_text(max_length=250)
    print(text)
    print('\n')

Model: BigramCounter
tied caw ball twitch steelkilt knowledge north try jump fields instantly sleeper chains seaman dust cable powers cod —a subtle pervading constructed bestow ice rampart house spoil others pitch beheld ambiguous paper heavenly st harem dropping after virtue downright descried distinct chance watery loftiest stay as caw thing proportions skull getting head hosea boys bestow glimpses unlike pointed written soundings forbearance power hackluyt ridge ascending bows supplied philosopher neither removed involved split streets rigging bristles drag assured waist atlantic meat never slightest ways east sound peleg empty pausing without horse sank rocks involutions rounding supper inns time fish diving yards sword harbors crowding their requiem dan only touching wharf insufferable discovered live inward cheerily privilege chap safe — arched darkness plane perceived last attention easy there once address animals murmured truly box beale englishman pursuit thou rest flesh opini

## Commento finale prestazioni modelli bigrammi e trigrammi

Abbiamo generato automaticamente testi basandoci su modelli linguistici n-grams (in particolare bigrammi e trigrammi). Quello che si evince, in particolare, riguarda il fatto che il modello basato sui bigrammi mantiene una buona coerenza linguistica, sia per l'italiano che per l'inglese, e le parole scelte provengono in maggior parte da un lessico socio-politico. Inoltre, il valore di perplexity relativo ai bigrammi ci suggerisce come il modello si adatta molto bene al linguaggio utilizzato. Al contrario, la situazione per i trigrammi è più complicata: la forte presenza di token UNKNOWN fa sì che ci riferiamo a un dizionario esterno, per cui il linguaggio utilizzato sarà meno attinente all'ambito socio-politico, e dunque più generale.